## Final Crawler (headless) _ V1

In [28]:
from selenium import webdriver
import time
import numpy as np
import pandas as pd

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
options.add_argument("lang=ko_KR") # 한국어!
# 혹은 options.add_argument("--disable-gpu")


# UserAgent값을 바꿔줌
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")

driver = webdriver.Chrome('C:/driver/chromedriver')#,chrome_options=options)
driver.get('https://manage.searchad.naver.com/customers/1488201/tool/keyword-planner')

time.sleep(3)

btn = driver.find_element_by_css_selector('body > elena-root > elena-home > div > div > div > elena-login-box > div > div.card.ng-star-inserted > div.card-body > div > a')
btn.click()

time.sleep(3)

log_id = driver.find_element_by_id('uid')
log_id.send_keys('myid')

log_pw = driver.find_element_by_id('upw')
log_pw.send_keys('mypw')

time.sleep(1)

login_btn = driver.find_element_by_css_selector('#container > div > div > fieldset > div > span > button')
login_btn.click()

time.sleep(3)

driver.get('https://manage.searchad.naver.com/customers/1488201/tool/keyword-planner')

time.sleep(1)

keywords_list = ['부동산','gㄹ토ㄷㅈ','부동산시세','데이터']

final_result = []
error_log = []

for i in keywords_list:
     
    keyword=driver.find_element_by_css_selector('body > elena-root > elena-wrap > div > elena-tool > div > div > div > div > elena-keyword-planner > div.row.lang-ko-KR > div.col-sm-9.col-keyword-query > div:nth-child(1) > div.card-body.hint-area > div > div > div:nth-child(1) > div > textarea')
    keyword.send_keys(i)

    
    lookupbtn=driver.find_element_by_css_selector('body > elena-root > elena-wrap > div > elena-tool > div > div > div > div > elena-keyword-planner > div.row.lang-ko-KR > div.col-sm-9.col-keyword-query > div:nth-child(1) > div.card-footer > button')
    lookupbtn.click()

    time.sleep(3)

    
    lookuplist=driver.find_elements_by_css_selector('body > elena-root > elena-wrap > div > elena-tool > div > div > div > div > elena-keyword-planner > div.row.lang-ko-KR > div.col-sm-9.col-keyword-query > div:nth-child(2) > div.elena-table-area > elena-table > div > div > table > tbody > tr')

    #lookuplist[0].find_elements_by_css_selector('td')[1].find_element_by_css_selector('span > span').click()
    lookuplist[0].find_elements_by_css_selector('td')[1].find_element_by_css_selector('#wgt-undefined > td.elenaColumn-relKeyword > elena-keyword').click()
    time.sleep(3)

    try:
        source=driver.find_element_by_css_selector('#wrap > div.modal.tool-keyword-planner-chart-controller.fade.ng-scope.ng-isolate-scope.in > div > div > div > div.modal-body > div > table > tbody > tr:nth-child(1) > td > div > div > table > tbody > tr > td > highchart')

        yaxis=source.find_elements_by_css_selector('svg > g.highcharts-axis-labels.highcharts-yaxis-labels > text')
        yaxis[0].text, yaxis[-1].text

        minmax=[]
        minmax.append(yaxis[0].text)
        minmax.append(yaxis[-1].text)

        int_minmax=[]
        for num in minmax :
            num=num.replace('k','000').replace(',','')
            if '.' in num :
                num=num.replace('.','')
                num=int(num)//10
            int_minmax.append(int(num))


        desktop = source.find_element_by_css_selector('#wrap > div.modal.tool-keyword-planner-chart-controller.fade.ng-scope.ng-isolate-scope.in > div > div > div > div.modal-body > div > table > tbody > tr:nth-child(1) > td > div > div > table > tbody > tr > td > highchart> div > svg > g.highcharts-series-group > g.highcharts-series.highcharts-series-0.highcharts-line-series > path.highcharts-graph')

        mobile=driver.find_element_by_css_selector('svg > g.highcharts-series-group > g.highcharts-series.highcharts-series-1.highcharts-line-series > path.highcharts-graph')
        mobile.get_attribute('d').split(' L')

        d_y_coords=[]
        m_y_coords=[]
        for line in desktop.get_attribute('d').split(' L') :

            d_y_coords.append(line.split()[-1])
        for line in mobile.get_attribute('d').split(' L') :
            m_y_coords.append(line.split()[-1])

        d_searchcount=[]
        m_searchcount=[]
        for y in d_y_coords :
            d_searchcount.append(int_minmax[-1] - (int_minmax[-1]-int_minmax[0])/336*float(y))
        for y in m_y_coords :
            m_searchcount.append(int_minmax[-1] - (int_minmax[-1]-int_minmax[0])/336*float(y))

        desktop_cnt=[]
        mobile_cnt=[]
        for cnt in d_searchcount :
            desktop_cnt.append(int(np.round(cnt)))
        for cnt in m_searchcount :
            mobile_cnt.append(int(np.round(cnt)))

        # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 뽑고 싶은 '월'  선택 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        final_result.append([i, desktop_cnt[-1], mobile_cnt[-1]])
        print('"{}" 키워드 정보 수집에 성공했습니다.'.format(i))

        closebtn=driver.find_element_by_css_selector('#wrap > div.modal.tool-keyword-planner-chart-controller.fade.ng-scope.ng-isolate-scope.in > div > div > div > div.modal-header > div.pull-right.ng-scope > button > i')
        closebtn.click()

        keyword.clear()
        time.sleep(2)
        
    except:
        error_log.append(i)
        print('>>>>>> Error 발생 : {}'.format(i))
        keyword.clear()
        time.sleep(2)

print('-----------------------------------------------------')
print('모든 키워드 수집에 성공했습니다.')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"span > span"}
  (Session info: chrome=75.0.3770.142)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.17763 x86_64)
